In [252]:
import numpy as np
from scipy import stats

In [320]:
parameter_set = {
    'theta': {
        'values': np.array([0.1, 0.5, 1, 3]),
        'prior': np.array([0.1, 0.5, 0.2, 0.2]),
        'dependencies_as_bool': np.array([0, 1, 0, 0, 0, 0, 0, 0]),
        'dependencies_as_str': np.array(['sigma'], dtype=object),
        'type': 'no_link'
    },
    'sigma': {
        'values': np.array([0.5, 1, 3, 6]),
        'prior': np.array([0.1, 0.1, 0.6, 0.2]),
        'dependencies_as_bool': np.array([1, 0, 0, 0, 0, 0, 0, 0]),
        'dependencies_as_str': np.array(['theta'], dtype=object),
        'type': 'no_link'
    },
    '0->1': {
        'values': np.array([-1, -1/2, 0, 1/2, 1]),
        'prior': np.array([0.5, 0.3, 0.1, 0.05, 0.05]),
        'dependencies_as_bool': np.array([1, 1, 0, 0, 0, 0, 0, 1]),
        'dependencies_as_str': np.array(['sigma', 'theta', '2->1'], dtype=object),
        'type': 'link'
    },
    '0->2': {
        'values': np.array([-1, -1/2, 0, 1/2, 1]),
        'prior': np.array([0.5, 0.3, 0.1, 0.05, 0.05]),
        'dependencies_as_bool': np.array([1, 1, 0, 0, 0, 1, 0, 0]),
        'dependencies_as_str': np.array(['sigma', 'theta', '1->2'], dtype=object),
        'type': 'link'
    },
    '1->0': {
        'values': np.array([-1, -1/2, 0, 1/2, 1]),
        'prior': np.array([0.5, 0.3, 0.1, 0.05, 0.05]),
        'dependencies_as_bool': np.array([1, 1, 0, 0, 0, 0, 1, 0]),
        'dependencies_as_str': np.array(['sigma', 'theta', '2->0'], dtype=object),
        'type': 'link'
    },
    '1->2': {
        'values': np.array([-1, -1/2, 0, 1/2, 1]),
        'prior': np.array([0.5, 0.3, 0.1, 0.05, 0.05]),
        'dependencies_as_bool': np.array([1, 1, 0, 1, 0, 0, 0, 0]),
        'dependencies_as_str': np.array(['sigma', 'theta', '0->2'], dtype=object),
        'type': 'link'
    },
    '2->0': {
        'values': np.array([-1, -1/2, 0, 1/2, 1]),
        'prior': np.array([0.5, 0.3, 0.1, 0.05, 0.05]),
        'dependencies_as_bool': np.array([1, 1, 0, 0, 1, 0, 0, 0]),
        'dependencies_as_str': np.array(['sigma', 'theta', '1->0'], dtype=object),
        'type': 'link'
    },
    '2->1': {
        'values': np.array([-1, -1/2, 0, 1/2, 1]),
        'prior': np.array([0.5, 0.3, 0.1, 0.05, 0.05]),
        'dependencies_as_bool': np.array([1, 1, 1, 0, 0, 0, 0, 0]),
        'dependencies_as_str': np.array(['sigma', 'theta', '0->1'], dtype=object),
        'type': 'link'
    }
}

In [255]:
def construct_parameter_combinations(param_labels, param_values):
    num_values = np.array([p_v.size for p_v in param_values])
    c = len(param_labels) # Number of parameters (columns)
    r = num_values.prod() # Number of tuples (rows)
    S = np.zeros((r, c))
    for i in range(c):
        tile_coef = int(num_values[i+1:].prod())
        rep_coef = int(num_values[:i].prod())
        ou = np.tile(param_values[i], (int(tile_coef), 1)).flatten('F')
        o = ou[np.tile(np.arange(ou.size), (1, rep_coef)).flatten()]
        S[:, i] = o.T
    return S


def construct_link_matrix(K):
    G = np.empty((K, K), dtype=object)
    for i in range(K):
        for j in range(K):
            if i == j:
                G[i, j] = ''
            else:
                G[i, j] = f'{i}->{j}'
    
    return G

In [289]:
K = 3
action = 2
action_bool = np.zeros(K, dtype=bool)
action_bool[action] = 1
print(action)
links_c = link_name_matrix[:, ~action_bool]
print(links_c)
#print(~action_bool)
#print(np.where(links_c == ''))
additional_dependencies = links_c[action_bool, :].flatten()
print(additional_dependencies)
print(links_c[~action_bool, :])
np.concatenate((additional_dependencies, np.delete(links_c[~action_bool, :], [0, links_c[~action_bool, :].size-1])))

2
[['' '0->1']
 ['1->0' '']
 ['2->0' '2->1']]
['2->0' '2->1']
[['' '0->1']
 ['1->0' '']]


array(['2->0', '2->1', '0->1', '1->0'], dtype=object)

In [314]:
action = 0
dep_bool = np.zeros(link_name_matrix.shape, dtype=bool)
for i in range(K):
    for j in range(K):
        if i != j and j != action:
            dep_bool[i, j] = 1

additional_dependencies = link_name_matrix[dep_bool]
additional_dependencies

array(['0->1', '0->2', '1->2', '2->1'], dtype=object)

In [315]:
dep_bool

array([[False,  True,  True],
       [False, False,  True],
       [False,  True, False]])

In [338]:
K = 3
dt = 1/5

belief_name = 'sigma'
belief_values = parameter_set[belief_name]['values']
link_name_matrix = construct_link_matrix(K)

action = 1
parameter_names = parameter_set[belief_name]['dependencies_as_str']
dep_bool = np.zeros(link_name_matrix.shape, dtype=bool)
for i in range(K):
    for j in range(K):
        if i != j and j != action:
            dep_bool[i, j] = 1

additional_dependencies = link_name_matrix[dep_bool]
additional_dependencies
parameter_names = np.concatenate((parameter_names, additional_dependencies))
param_values_list = [parameter_set[name]['values'] for name in parameter_names]
parameter_values = construct_parameter_combinations(parameter_names, param_values_list)
var_to_consider= np.ones(K, dtype=bool)
var_to_consider[action] = 0

X = np.array([40, 21, -12])
X_prev = np.array([35, 20, -5])

out = np.zeros((belief_values.size, parameter_values.shape[0]))
for k in range(K):
    if var_to_consider[k] == 0:
        continue
    
    current_var = np.zeros(K).astype(bool)
    current_var[k] = 1
    
    if belief_name == 'sigma':
        thetas = parameter_values[:, parameter_names.tolist().index('theta')]
        sigmas = belief_values
    else:
        sigmas = parameter_values[:, parameter_names.tolist().index('sigma')]
        thetas = belief_values

    links_indices = [parameter_names.tolist().index(param) for param in parameter_names[1:] if int(param[-1]) == k]
    #print(links_indices)
    links = np.ones((K, parameter_values.shape[0]))
    links[~current_var, :] = parameter_values[:, links_indices].T

    regularisor = -1 * X_prev[k] * (np.abs(X_prev[k]) / 100)
    
    if belief_name == 'theta':
        mus = thetas.reshape((thetas.size, 1)) * (X_prev @ links + regularisor - X_prev[k]) * dt
        out += stats.norm.logpdf(X[k], loc=mus, scale=sigmas)
    else:
        mus = thetas * (X_prev @ links + regularisor - X_prev[k]) * dt
        out += stats.norm.logpdf(X[k], loc=mus, scale=sigmas.reshape((sigmas.size, 1)))

In [339]:
out

array([[-3526.08368271, -3534.19768271, -3542.32168271, ...,
        -6576.74158271, -6793.34158271, -7018.94158271],
       [ -883.24590207,  -885.27440207,  -887.30540207, ...,
        -1645.91037707, -1700.06037707, -1756.46037707],
       [ -101.96932664,  -102.19471553,  -102.4203822 , ...,
         -186.70982387,  -192.72649053,  -198.9931572 ],
       [  -29.90495225,   -29.96129948,   -30.01771614, ...,
          -51.09007656,   -52.59424323,   -54.16090989]])

In [354]:
Qs = np.array([np.array([0, 1, 2]), np.array([0, 1, 2, 3, 4]), np.array([0, 1, 2, 3, 4]), np.array([0, 1, 2, 3, 4])], dtype=object)
select = np.array([0, 1, 1, 1], dtype=bool)

np.vstack(Qs[select])

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])